## RAG 구현
검색 -> 증강 -> 생성(LLM을 빌려서 수행)

In [ ]:
#langchain과 관련된 베이스 라이브러리들.
!pip install langchain openai langchain-community


#tiktoken은 언어의 '토큰화'를 지원해줌
!pip install tiktoken


#bm25는 벡터의 유사도 검색을 잘 해내는 라이브러리임
!pip install rank_bm25


#트랜스포머를 기반으로 문장이나 문서를 임베딩할 수 있도록 돕는 라이브러리
!pip install sentence-transformers


#벡터 데이터를 저장함(벡터 DB)
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.3
    Uninstalling langchain-core-1.1.3:
      Successfully uninstalled langchain-core-1.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is inc

In [ ]:
import tiktoken, openai

#'임베딩'을 담당하는 라이브러리.
#허깅페이스의 특정 모델을 기반으로 임베딩을 도와주는 것이 HuggingFace~~~Embeddings 임
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings

#문서를 가져온 후, 문서를 일정한 길이로 잘라줌(Text->Splitter)
from langchain.text_splitter import RecursiveCharacterTextSplitter

#VectorDB. 데이터를 임베딩한 후 숫자로 변한 데이터를 저장하고,
#DB내에서 숫자 데이터의 검색을 더 빠르게 할 수 있도록 지원함
from langchain.vectorstores import Chroma
#from langchain_community.vectorstores import FAISS > cpu용 FAISS, gpu용 FAISS

#pdf 파일이나 문서 파일을 입력했을 때, 내부의 데이터를 가져옴
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

#웹사이트 텍스트를 가져오는 역할
from langchain.document_loaders import WebBaseLoader

ModuleNotFoundError: No module named 'langchain.embeddings.sentence_transformer'

In [ ]:
#토크나이저는 -> 토큰화를 시키는 것
#'cl100k_base' -> 토크나이저의 종류, gpt가 사용하는 토크나이저와 유사한 특징을 가짐
tokenizer = tiktoken.get_encoding('cl100k_base')


#인풋된 text가 정해진 토크나이저로 잘려 나가면, 그 길이를 알 수 있게 됨
def titoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)


#모델 다운로드
model_hug = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')

In [ ]:
loaders=[
    WebBaseLoader("https://github.com/huggingface"),
    WebBaseLoader("https://wikidocs.net/229987"),
    WebBaseLoader("https://en.wikipedia.org/wiki/Hugging_Face"),
]

In [ ]:
#데이터를 토큰화 하여 load 하기 위한 내용
docs = []

for load in loaders:
  # ~~Loader로 가져온 데이터에 대해 데이터를 불러오고 가져옴 : load_and_split()
  docs.extend(load.load_and_split())

print(f'총 문서의 길이 : {len(docs)}')

In [ ]:
docs[0]

In [ ]:
docs[15].page_content

In [ ]:
#토큰화(청크화)
#chunk_size = 문서(Document)를 얼마 단위로 자를까?
#chunk_overlap
#나는 코딩하는 것을 좋아합니다.
#나는 코딩하는 것을
#         하는 것을 좋아합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 250,
                                               chunk_overlap = 20,
                                               length_function = titoken_len)

texts = text_splitter.split_documents(docs)
print(f'자른 내용 : {texts[0]}, 총 길이 {len(texts)}')

In [ ]:
from langchain.retrievers import BM25Retriever

#BM25 TF-IDF(문서 빈도-역빈도)계열의 준수한 성능을 보이는 알고리즘
bm25_r = BM25Retriever.from_documents(texts)

#질문과 가장 닮은 내용을 담고 있는 top 3개를 뽑아라~
bm25_r.k = 3

In [ ]:
result = bm25_r.invoke('What is the huggingface?')
result[0].page_content

In [ ]:
result = bm25_r.invoke('What is the huggingface?')
result[1].page_content

In [ ]:
result = bm25_r.invoke('What is the huggingface?')
result[2].page_content

In [ ]:
import os
from google.colab import userdata

os.environ['OPEN_API_KEY'] = userdata.get('ssu')

#리트리버 관련 모듈
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

In [ ]:
#채팅할 대상인 llm 모델 객체 만듦
llm = ChatOpenAI()

#우리가 잘라 둔 text를 VectorDB에 넣어줌
#왜? 벡터(숫자)로 만들고 빠르게 '계산' 할 수 있게 하기 위해서 => '유사도' 검색
#Chroma dB에 texts를 model_hug 방법으로 임베딩해서, 넣어놓음
chroma_db = Chroma.from_documents(texts, model_hug)

In [ ]:
#리트리버를 정의
multi_query = MultiQueryRetriever.from_llm(retriever=chroma_db.as_retriever(),
                                           llm=llm)

response = multi_query.invoke('What is the huggingface?')
print(response)


In [ ]:
from langchains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, #이야기를 나눔 llm
                                 chain_type = 'studff', #
                                retriever=multi_r)

response = qa('허깅페이스가 뭐야?')
print(response['result'])